# Data Analysis Question for FINRA

<br>
<br>

### Configuration

In [2]:
import os

import pandas as pd
from pandasql import sqldf
# nice tutorial: http://blog.yhat.com/posts/pandasql-sql-for-pandas-dataframes.html

In [34]:
# convenience wrapper
def pysqldf(q):
    return sqldf(q, globals())

<br>
<br>

### Data import cleaning

In [6]:
file = os.getcwd() + '/Data/trades/trades.csv'
trades = pd.read_csv(file)
trades.shape

(10, 6)

In [24]:
trades.loc[trades.FIRM == 'CDE'] = '3CDE'   # I assume this is incorrect, but I was not provided any guidance

<br>
<br>

### Descriptive analysis

In [35]:
q  = """
SELECT *, 
    CASE SIDE
        WHEN 'B' THEN QUANTITY * PRICE
        ELSE -1 * QUANTITY * PRICE
        END SIZE
FROM trades
ORDER BY FIRM, DATE;
"""
trades_1 = pysqldf(q)
trades_1

,DATE,FIRM,SYMBOL,SIDE,QUANTITY,PRICE,SIZE
0,02/03/14,1ABC,A123,B,200,41,8200
1,02/07/14,1ABC,C345,S,600,70,-42000
2,02/21/14,1ABC,A123,B,300,40,12000
3,02/24/14,1ABC,A123,S,300,30,-9000
4,02/04/14,2BCD,B234,B,600,60,36000
5,02/10/14,3CDE,C345,S,600,70,-42000
6,02/14/14,3CDE,B234,B,300,61,18300
7,02/27/14,3CDE,B234,S,1100,63,-69300
8,02/12/14,4DEF,B234,B,200,62,12400
9,02/25/14,4DEF,C345,B,2100,71,149100


In [36]:
q  = """
SELECT FIRM, SYMBOL, SUM(SIZE) as TOTAL
FROM trades_1
GROUP BY FIRM, SYMBOL;
"""
pysqldf(q)

,FIRM,SYMBOL,TOTAL
0,1ABC,A123,11200
1,1ABC,C345,-42000
2,2BCD,B234,36000
3,3CDE,B234,-51000
4,3CDE,C345,-42000
5,4DEF,B234,12400
6,4DEF,C345,149100


<br>
<br>

### Questions

1. Based on the data in the table above, please draft a paragraph that describes your data observations and description of what is going on in the market during February 2014.

There appear to be a few interesting trades that are made in the month of February 2014.  Firm 4DEF bought a relatively large amount of security C345, near the end of the month.  That might be indicative of an insider tip.

In addition, firm 3CDE sold the exact same amount of security C345, as firm 1ABC, 3 days after 1ABC sold.  I don't have enough information to understand what schema they may be playing, immediately, but I would investigate, further.

<br>
2. Your business user asks you for a series of items as follows. In each of the items below, if you make assumptions to complete the task, please document them. 

A. Your business user asks you to tell them the unique symbols in the data table above.

* Please write the SQL query you would use to query this table.
* Please show the exact results you expect based on your SQL query.

In [38]:
q  = """
SELECT DISTINCT(SYMBOL) as Unique_Symbols
FROM trades_1;
"""
pysqldf(q)

,Unique_Symbols
0,A123
1,C345
2,B234


<br>
B. Your business user asks you to tell them the unique firm and symbol combinations in the data table above.

* Please write the SQL query you would use to query this table.
* Please show the exact results you expect based on your SQL query.

In [44]:
q  = """
SELECT DISTINCT( FIRM ||'-'|| SYMBOL ) as Combination
FROM trades_1;
"""
pysqldf(q)

,Combination
0,1ABC-A123
1,1ABC-C345
2,2BCD-B234
3,3CDE-C345
4,3CDE-B234
5,4DEF-B234
6,4DEF-C345


<br>

C. Your business user asks you to show them a table that includes the number of trades for each firm and symbol combination in the data table above.

* Please write the SQL query you would use to query this table.
* Please show the exact results you expect based on your SQL query.

In [47]:
q  = """
SELECT Combination, Count(*) Count
FROM (
SELECT FIRM ||'-'|| SYMBOL as Combination
FROM trades_1
)
GROUP BY Combination
;
"""
pysqldf(q)

,Combination,Count
0,1ABC-A123,3
1,1ABC-C345,1
2,2BCD-B234,1
3,3CDE-B234,2
4,3CDE-C345,1
5,4DEF-B234,1
6,4DEF-C345,1


<br>

3. Based on the data table above, your business user asks you to create a report that shows, for each firm, the symbols they traded and the total dollar volume they traded in that symbol.

A. Please write the business requirements and corresponding functional specifications for this business request in the following format. 

BR #    |    Business Requirement    |    Functional Specification

1 | Create calculation of total dollar volume for each trade | Create one new column with QUANTITY * PRICE, multiplied by (1) if it is a buy, and (-1) if it is a sell

2 | Create a report for each firm, each symbol they traded and associated total volume amount | Create new query by grouping on FIRM and SYMBOL within FIRM.  Include the sum of all trades for that grouping

B. If you have clarifying questions, please document them, make assumptions to complete your task, and document your assumptions.

There was no guidance on time period.  I will assume that the time period is February 2014, because that is the only data I have.

<br>
<br>

Below is the sql impelementation

In [35]:
q  = """
SELECT *, 
    CASE SIDE
        WHEN 'B' THEN QUANTITY * PRICE
        ELSE -1 * QUANTITY * PRICE
        END SIZE
FROM trades
ORDER BY FIRM, DATE;
"""
trades_1 = pysqldf(q)
trades_1

,DATE,FIRM,SYMBOL,SIDE,QUANTITY,PRICE,SIZE
0,02/03/14,1ABC,A123,B,200,41,8200
1,02/07/14,1ABC,C345,S,600,70,-42000
2,02/21/14,1ABC,A123,B,300,40,12000
3,02/24/14,1ABC,A123,S,300,30,-9000
4,02/04/14,2BCD,B234,B,600,60,36000
5,02/10/14,3CDE,C345,S,600,70,-42000
6,02/14/14,3CDE,B234,B,300,61,18300
7,02/27/14,3CDE,B234,S,1100,63,-69300
8,02/12/14,4DEF,B234,B,200,62,12400
9,02/25/14,4DEF,C345,B,2100,71,149100


In [36]:
q  = """
SELECT FIRM, SYMBOL, SUM(SIZE) as TOTAL
FROM trades_1
GROUP BY FIRM, SYMBOL;
"""
pysqldf(q)

,FIRM,SYMBOL,TOTAL
0,1ABC,A123,11200
1,1ABC,C345,-42000
2,2BCD,B234,36000
3,3CDE,B234,-51000
4,3CDE,C345,-42000
5,4DEF,B234,12400
6,4DEF,C345,149100
